#### For generating HTML tags and JSON containing all the recipes

In [ ]:
import pandas as pd
import subprocess

In [ ]:
recipe_entry_pug = '''tr
  .accordion-item
    h2.accordion-header
      button.accordion-button.collapsed(type='button' data-bs-toggle='collapse' data-bs-target='#collapse{id}' aria-expanded='false' aria-controls='collapse{id}')
        strong {recipe_name}
    #collapse{id}.accordion-collapse.collapse(data-bs-parent='#accordionExample')
      .accordion-body
        div
          .attributes
            {type}
            span.px-1 |
            {ingredients}
            span.px-1 |
            {effects}
          p {steps}
'''

type_tag = 'span.badge.rounded-pill.text-bg-primary {type}'

# also amount?
ingredient_tag = 'span.badge.rounded-pill.text-bg-dark {ingredient}'

effect_tag = 'span.badge.rounded-pill.text-bg-success {effect}'

first_level_item = '''li
  a.dropdown-item(href='#') {category}
  ul.dropdown-menu.dropdown-submenu
    {second_level_items}'''

checklist_pug = '''li
      .form-check
        input#{type}_{id}.form-check-input(type='checkbox' value='{item}')
        label.form-check-label.ml-3(for='ingredient_{id}') {item}
'''

### For generating category

In [ ]:
checks = []

category = pd.read_csv('./categorize/category_eng.csv', sep='|')

for i, c in enumerate(category['category_en'].unique()):
    checks.append(checklist_pug.format(type='category', id=i, item=c))

with open('category_checks.pug', 'w') as f:
    f.write(''.join(checks))

# exec pug with generated tags
subprocess.run(['pug', 'category_checks.pug'], stderr=subprocess.DEVNULL)

### For generating effects

In [ ]:
checks = []

effect_category = pd.read_csv('./categorize/effect_category_eng.csv', sep='|')

for i, c in enumerate(effect_category['category'].unique()):
    checks.append(checklist_pug.format(type='effect', id=i, item=c))

with open('effect_checks.pug', 'w') as f:
    f.write(''.join(checks))

# exec pug with generated tags
subprocess.run(['pug', 'effect_checks.pug'], stderr=subprocess.DEVNULL)

### For generating ingredients checkboxes

In [ ]:
checks = []

ingredient_category = pd.read_csv('./categorize/ingredient_category_eng.csv', sep='|')

for i, c in enumerate(ingredient_category['category'].unique()):
    ingred = []
    for j, ingredient in enumerate(ingredient_category[ingredient_category['category'] == c]['Ingredient_en']):
        ingredient = ' '.join(ingredient.split('_')).title()
        ingred.append(checklist_pug.format(type='ingredient', id=f'{i}_{j}', item=ingredient))
    second_level = '    '.join(ingred)
    checks.append(first_level_item.format(category=' '.join(c.split('_')).title() + ' &raquo;', second_level_items=second_level))

with open('ingredient_checks.pug', 'w') as f:
    f.write(''.join(checks))

# exec pug with generated tags
subprocess.run(['pug', 'ingredient_checks.pug'], stderr=subprocess.DEVNULL)

### For generating methods

In [ ]:
checks = []

methods = pd.read_csv('./categorize/methods.csv')

list(methods['method'].unique())
method_translated = ['Boil', 'Simmer', 'Steam', 'Pan-fry', 'Bake', 'Braise', 'Broil', 'Stir-fry', 'Roast', 'Deep-fry']
method_mapping = {}

for i, e in enumerate(list(methods['method'].unique())):
    method_mapping[e] = method_translated[i]

for i, c in enumerate(methods['method'].unique()):
    checks.append(checklist_pug.format(type='method', id=i, item=c, item_translated=method_mapping[c]))

with open('method_checks.pug', 'w') as f:
    f.write(''.join(checks))

# exec pug with generated tags
subprocess.run(['pug', 'method_checks.pug'], stderr=subprocess.DEVNULL)

### For generating JSON for receipe searching

In [ ]:
# for each recipe, generate a pug entry
entries = []

recipes = pd.read_csv('./full_recipe.csv', sep='|')

category = pd.read_csv('./categorize/category_eng.csv', sep='|')

ingredients = pd.read_csv('./ingredient/all_ingredient.csv', sep='|')

effects = pd.read_csv('./categorize/effect_eng.csv', sep='|')

method= pd.read_csv('./categorize/methods_eng.csv', sep='|')

ingredient_categories= pd.read_csv('./categorize/ingredient_category_eng.csv', sep='|')

effect_ch= pd.read_csv('./categorize/effect.csv', sep=',')


for name in recipes['Food_Name_en'].unique():
    entry = {}
    recipe_name = name
    recipe_name_ch=recipes[recipes['Food_Name_en'] == recipe_name]['Food_Name'].values[0]
    type = category[category['Food_Name_en'] == recipe_name]['category_en'].values[0]
    food_ingredients = list(ingredients[ingredients['Food_Name_en'] == recipe_name]['Ingredient_en'].values)
    food_effects = list(effects[effects['Food_Name_en'] == recipe_name]['Effect_en'].values)
    food_effect_categories = list(effect_ch[effect_ch['Food_Name'] == recipe_name_ch]['Category'].unique())
    cook_method=list(method[method['Food_Name_en'] == recipe_name]['method_en'].values)
    food_steps = recipes[recipes['Food_Name_en'] == recipe_name]['Steps_en'].values
    food_steps = food_steps[0] if len(food_steps) > 0 else ''
    food_steps = food_steps if food_steps == food_steps else ''
    entry['recipe_name'] = recipe_name
    entry['category'] = type
    entry['ingredients'] = food_ingredients
    entry['effects'] = food_effects
    entry['steps'] = food_steps
    entry['method'] = cook_method
    entry['effect_category'] = food_effect_categories
    entries.append(entry)

import json
with open('recipes.json', 'w', encoding='utf-8-sig') as f:
    json.dump(entries, f, ensure_ascii=False, indent=4)